<a href="https://colab.research.google.com/github/saurabhkirar/NLP_END/blob/main/Session8/Session8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Part1 
### Below is the code to convert the ENglish  sentence into German using pytorch and sequence2seq model .

```
# This is formatted as code
```



In [1]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=4cce60dc48a36568daad9255256a96b153958176f9c1d9cfdccabb1cb8ee9f8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-qg7ywasj/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [2]:
## Get the necessary liabraries.
import torch
import torch.optim as optim
import torch.nn as nn
from torchtext.datasets import Multi30k
from torchtext.data import Field,BucketIterator
import spacy
import numpy as np
import random,math,time

In [3]:
SEED=99
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [5]:
## define the tokenization function to convert the text to tokens.
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
## Need to define fields to construct how the data shoule be processed, two fields will be defines one
## for the source(en) and one for the target(de)

In [6]:
SRC= Field(tokenize=tokenize_en, init_token = '<sos>', eos_token = '<eos>', lower = True)
TRG= Field(tokenize=tokenize_de, init_token = '<sos>', eos_token = '<eos>', lower = True)

In [7]:
## Once the fileds are defined time to load the data set and do a split 
train_data, valid_data, test_data=Multi30k.splits(exts=('.en','.de'),fields=(SRC,TRG))

training.tar.gz:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.09MB/s]
validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 1.49MB/s]

downloading validation.tar.gz
downloading mmt_task1_test2016.tar.gz



mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 1.33MB/s]


In [8]:
## After the data set is loaded the next step is to build the vocabulary for english and german language.
## here also we want to keep only those words which have appeared atleast two times.
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (de) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (en) vocabulary: 5893
Unique tokens in target (de) vocabulary: 7855


In [9]:
## Need to set the device as GPU if the device is available, we need to pass this device to the iterators.
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device is ',device)

device is  cuda


In [10]:
## the final step of data prep is to prepare the iterators, here we will be using the bucket iterators
## itertaors loop throgh the data and gove us the tokenize values of the text to be processed.
## how much data is governed by the batch size and padding is included in the iterators themselves.
## bucket iterators helps us to put the similar lenght text together thus minimizing the padding efforts in a batch.
BATCH_SIZE=128
train_iterator,valid_iterator,test_iterator=BucketIterator.splits((train_data,valid_data,test_data),batch_size=BATCH_SIZE,device=device)

In [11]:
## define a class Encoder.

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        # Here you can use nn.GRU(emb_dim, hid_dim) or 
        # nn.RNN(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [12]:
## after the fwd function is defined, thenext step is to define a decoder function which decodes th esentence.
## decoder works and decodes each word per time stamp.We have couple of addtional objects one is the output_dim and other is a linear layer to do the predictions from the top layer hidden state.
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [14]:
## Training the Seq2Seq model.
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [15]:
## Weight Initialization.
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,905,519 trainable parameters


In [17]:
## next step is to define the optimizer.
optimizer = optim.Adam(model.parameters())

In [18]:
## Next step is to define a loss function.
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [19]:
## define a training function which wil train the model.


def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [20]:
## function to evaluate the model.
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [21]:
## function to calculat the epochs.
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [22]:
## actual model training.
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 36s
	Train Loss: 5.190 | Train PPL: 179.426
	 Val. Loss: 5.033 |  Val. PPL: 153.423
Epoch: 02 | Time: 0m 35s
	Train Loss: 4.688 | Train PPL: 108.669
	 Val. Loss: 5.016 |  Val. PPL: 150.733
Epoch: 03 | Time: 0m 35s
	Train Loss: 4.383 | Train PPL:  80.118
	 Val. Loss: 4.940 |  Val. PPL: 139.715
Epoch: 04 | Time: 0m 35s
	Train Loss: 4.114 | Train PPL:  61.192
	 Val. Loss: 4.713 |  Val. PPL: 111.409
Epoch: 05 | Time: 0m 35s
	Train Loss: 3.874 | Train PPL:  48.125
	 Val. Loss: 4.680 |  Val. PPL: 107.762
Epoch: 06 | Time: 0m 36s
	Train Loss: 3.663 | Train PPL:  38.994
	 Val. Loss: 4.520 |  Val. PPL:  91.799
Epoch: 07 | Time: 0m 35s
	Train Loss: 3.452 | Train PPL:  31.571
	 Val. Loss: 4.314 |  Val. PPL:  74.708
Epoch: 08 | Time: 0m 35s
	Train Loss: 3.280 | Train PPL:  26.570
	 Val. Loss: 4.151 |  Val. PPL:  63.480
Epoch: 09 | Time: 0m 35s
	Train Loss: 3.116 | Train PPL:  22.547
	 Val. Loss: 4.079 |  Val. PPL:  59.111
Epoch: 10 | Time: 0m 35s
	Train Loss: 2.981 | Train PPL

## Part2 
Below are the 100 questions in python.

In [ ]:
# Write a pyth0n program to multiply two number 
num1=100.19
num2=200.43
num3=num1*num2
print('The product of %.2f and %.2f is %.2f'%(num1,num2,num3))

The product of 100.19 and 200.43 is 20081.08


In [ ]:
# Write a program to print the positive number of all the  given number in a list 
lis=[1,3,-9,-81,7776,-9876]
abs_list=[]
print('actual list',lis)
for rec in lis:
  rec=abs(rec)
  abs_list.append(rec)
print('list with absolute values',abs_list)

actual list [1, 3, -9, -81, 7776, -9876]
list with absolute values [1, 3, 9, 81, 7776, 9876]


In [ ]:
## Write a program that has a python function to return the value if the input  is greater than zero else the function should return zero
# test the function with one postive and one negetive value.
def func_relu(x):
  return x if x > 0 else 0

print('the output for 98 is ',func_relu(98))
print('the output for -98  is ',func_relu(-98))

the output for 98 is  98
the output for -98  is  0


In [ ]:
## write a python code to find the maximum and mimimum values in a list.
lst=[98,776,1,22,33]
print('Maximum value in the list is ',max(lst))
print('minimum vale in the list is ',min(lst))

Maximum value in the list is  776
minimum vale in the list is  1


In [ ]:
## Write a function in python which takes a list as an input and returns the normalized list as the output.
## x normalized = (x – x minimum) / (x maximum – x minimum)
## test the function and print the normalized list
def func_normalization(p_list):
  l_max=max(p_list)
  l_min=min(p_list)
  deno=l_max-l_min
  norm_lst=[]
  for rec in p_list:
    norm_lst.append((rec-l_min)/deno)
  return norm_lst

nat_list=[234,345,887,98]
func_normalization(nat_list)

[0.17237008871989862, 0.31305449936628643, 1.0, 0.0]

In [ ]:
## Write a code to print the  memory address of a given array 
from array import *
arr=array('i',[1,5,7,89])
print('Memory address of the given array is ',arr.buffer_info())

Memory address of the given array is  (140549508420864, 4)


In [ ]:
## Write a code to print the  no of element of a given array 
from array import *
arr=array('i',[1,5,7,89])
print('There are total %d elements '%(arr.buffer_info()[1]))

There are total 4 elements 


In [ ]:
#Write a Python program to remove the first occurrence of a specified element from an array
from array import *
array_num = array('i', [1, 3, 15, 3, 37, 21, 29, 3])
print("Original array: "+str(array_num))
array_num.remove(3)
print("New array: "+str(array_num))

Original array: array('i', [1, 3, 15, 3, 37, 21, 29, 3])
New array: array('i', [1, 15, 3, 37, 21, 29, 3])


In [ ]:
## write a code to define dictionary and loop over it.
d = {'x': 10, 'y': 20, 'z': 30} 
for dict_key, dict_value in d.items():
    print(dict_key,'->',dict_value)

x -> 10
y -> 20
z -> 30


In [ ]:
## Write a code to merge two dictionaries.
d1 = {'a': 100, 'b': 200}
d2 = {'x': 300, 'y': 200}
d = d1.copy()
d.update(d2)
print(d)

{'a': 100, 'b': 200, 'x': 300, 'y': 200}


In [ ]:
## write a code to merge two lists into one dictionary.
keys = ['sachin', 'hayden', 'sana']
values = ['Ind','Aus', 'SL']
player = dict(zip(keys, values))
print(player)

{'sachin': 'Ind', 'hayden': 'Aus', 'sana': 'SL'}


In [ ]:
## write a class in oythin to get the dictionary from an object field, test the code.
class Player(object):
     def __init__(self):
         self.ind = 'sachin'
         self.aus = 'hayden'
         self.sl = 'sana'
     def do_nothing(self):
         pass
test = Player()
print(test.__dict__)

{'ind': 'sachin', 'aus': 'hayden', 'sl': 'sana'}


In [ ]:
"""
Write a Python program to print all unique values in a dictionary. Go to the editor
Sample Data : [{"V":"1"}, {"V": "2"}, {"VI": "1"}, {"VI": "5"}, {"VII":"5"}, {"V":"9"},{"VIII":"7"}]
Expected Output : Unique Values: {'5', '2', '7', '1', '9'}
"""
L = [{"V":"1"}, {"V": "2"}, {"VI": "1"}, {"VI": "5"}, {"VII":"5"}, {"V":"9"},{"VIII":"7"}]
print("Original List: ",L)
u_value = set( val for dic in L for val in dic.values())
print("Unique Values: ",u_value)

Original List:  [{'V': '1'}, {'V': '2'}, {'VI': '1'}, {'VI': '5'}, {'VII': '5'}, {'V': '9'}, {'VIII': '7'}]
Unique Values:  {'2', '1', '7', '9', '5'}


In [ ]:
## Write a pythin code to add three lists using lambda and map.
nums1 = [1, 2, 3] 
nums2 = [4, 5, 6] 
nums3 = [7, 8, 9] 
print("Original list: ")
print(nums1)  
print(nums2)  
print(nums3)  
result = map(lambda x, y, z: x + y + z, nums1, nums2, nums3) 
print("\nNew list after adding above three lists:")
print(list(result))

Original list: 
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

New list after adding above three lists:
[12, 15, 18]


In [ ]:
## use time and datetine and list down few formats 
import time
import datetime
print("Current date and time: " , datetime.datetime.now())
print("Current year: ", datetime.date.today().strftime("%Y"))
print("Month of year: ", datetime.date.today().strftime("%B"))
print("Week number of the year: ", datetime.date.today().strftime("%W"))
print("Weekday of the week: ", datetime.date.today().strftime("%w"))
print("Day of year: ", datetime.date.today().strftime("%j"))
print("Day of the month : ", datetime.date.today().strftime("%d"))
print("Day of week: ", datetime.date.today().strftime("%A"))

Current date and time:  2020-12-16 15:35:07.374382
Current year:  2020
Month of year:  December
Week number of the year:  50
Weekday of the week:  3
Day of year:  351
Day of the month :  16
Day of week:  Wednesday


In [ ]:
## Write a code to show the unique file type based on the extension from a list.

lst=['a.jpg','b.png','c.jpeg','d.java','e.sql','f.sql','g.sql','z.png']
uniq=[]
for rec in lst:
    E =rec.split('.')[1]
    uniq.append(E)
set(uniq)  ## Set gives the distinct or non repetative values of the list.

{'java', 'jpeg', 'jpg', 'png', 'sql'}

In [ ]:
## Write a python function to return the greatest common divisor (GCD) for two numbers, test the functions.
def gcd(x, y):
    gcd = 1
    
    if x % y == 0:
        return y
    
    for k in range(int(y / 2), 0, -1):
        if x % k == 0 and y % k == 0:
            gcd = k
            break  
    return gcd

print(gcd(12, 17))
print(gcd(4, 6))

1
2


In [ ]:
## Write a code to calculate the euclidiean distance between two cordinates, express the cordinates  as list members.
import math
p1 = [4, 0]
p2 = [6, 6]
distance = math.sqrt( ((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2) )

print(distance)

6.324555320336759


In [ ]:
## Write a code to see if any file exists in the location.
import os.path
open('abc.txt', 'w')
print(os.path.isfile('abc.txt'))

True


In [ ]:
## Write a code to find out number of CPU using.
import multiprocessing
print(multiprocessing.cpu_count())

2


In [ ]:
## Wite a code to find out profiling of python programs.
import cProfile
def res():
    print(100.72*344.87)
cProfile.run('res()')

34735.3064
         39 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-17-a8dede55eddf>:3(res)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:382(write)
        3    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        3    0.000    0.000    0.000    0.000 socket.py:432(send)
        3    0.000    0.000    0.000    0.000 threading.py:1062(_wait_for_tstate_lock)
        3    0.000    0.000    0.000    0.000 threading.py:1104(is_alive)
        3    0.000    0.000    0.000    0.000 threading.py:506(is_set)
        1    0.0

In [ ]:
## Write a program to get the current user name 
import getpass
print(getpass.getuser())

root


In [ ]:
## Write a code to print the URL content to the console.
from http.client import HTTPConnection
conn = HTTPConnection("google.com")
conn.request("GET", "/")  
result = conn.getresponse()
# retrieves the entire contents.  
contents = result.read() 
print(contents)

b'<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">\n<TITLE>301 Moved</TITLE></HEAD><BODY>\n<H1>301 Moved</H1>\nThe document has moved\n<A HREF="http://www.google.com/">here</A>.\r\n</BODY></HTML>\r\n'


In [ ]:
## write a pyhon function to give the running total and test with with a list.
from itertools import accumulate
import operator
def running_sum(it):
    return accumulate(it,operator.add)

#List
result = running_sum([1,2,3,4,5,6,7])
print("Running sum of a list:")
for i in result:
    print(i)

Running sum of a list:
1
3
6
10
15
21
28


In [ ]:
#Write a Python program to check that a string contains only a certain set of characters (in this case a-z, A-Z and 0-9).
import re
def is_allowed_specific_char(string):
    charRe = re.compile(r'[^a-zA-Z0-9.]')
    string = charRe.search(string)
    return not bool(string)

print(is_allowed_specific_char("ABCDEFabcdef123450")) 
print(is_allowed_specific_char("*&%@#!}{"))

True
False


In [ ]:
#Write a Python program that iterate over elements repeating each as many times as its count.
from collections import Counter
c = Counter(p=4, q=2, r=0, s=-2)
print(list(c.elements()))


['p', 'p', 'p', 'p', 'q', 'q']


In [ ]:
## find the most common words in a sentence.
from collections import Counter
import re
text = """The start of T20 in a way revolutionised the cricket industry for good.  When we say this it is not just in terms of sports, fame, and viewership but also how it pumped up cricketer’s income. There are various leagues like BBL, IPL, CPL, PSL, and BPL which are pulling several big names across the globe which automatically helps in garnering massive revenue.

With the genesis of T20 cricket, cricket has witnessed revolutionary changes in terms of the popularity of the sport and the income earned by cricketers. The format has brought in a fast-moving, entertaining form of cricket to the world, forcing batsmen and bowlers to garner out-of-the-box solutions to counter the attacking prowess of the opponent.."""
words = re.findall('\w+',text)
print(Counter(words).most_common(10))

[('the', 10), ('of', 8), ('in', 5), ('cricket', 4), ('and', 4), ('to', 3), ('The', 2), ('T20', 2), ('a', 2), ('it', 2)]


In [ ]:
#Write a Python program to find the most common elements and their counts of a specified text.
from collections import Counter
s = 'lkseropewdssafsdfafkpwe'
print("Original string: "+s)
print("Most common three characters of the said string:")
print(Counter(s).most_common(3))

Original string: lkseropewdssafsdfafkpwe
Most common three characters of the said string:
[('s', 4), ('e', 3), ('f', 3)]


In [ ]:
## Write a code to merge two dictionaries 
import collections as ct
def merge_dictionaries(color1,color2):
    merged_dict = dict(ct.ChainMap({}, color1, color2))
    return merged_dict
color1 = { "R": "Red", "B": "Black", "P": "Pink" }
color2 = { "G": "Green", "W": "White" }
print("Original dictionaries:")
print(color1,' ',color2)
print("\nMerged dictionary:")
print(merge_dictionaries(color1, color2))

Original dictionaries:
{'R': 'Red', 'B': 'Black', 'P': 'Pink'}   {'G': 'Green', 'W': 'White'}

Merged dictionary:
{'W': 'White', 'G': 'Green', 'R': 'Red', 'P': 'Pink', 'B': 'Black'}


In [ ]:
## write a code to print count of each element in the list.
sampleList = [11, 45, 8, 11, 23, 45, 23, 45, 89]
print("Original list ", sampleList)

countDict = dict()
for item in sampleList:
  if(item in countDict):
    countDict[item] += 1
  else:
    countDict[item] = 1
  
print("Printing count of each item  ",countDict)

Original list  [11, 45, 8, 11, 23, 45, 23, 45, 89]
Printing count of each item   {11: 2, 45: 3, 8: 1, 23: 2, 89: 1}


In [ ]:
##Given a following two sets find the intersection and remove those elements from the first set
firstSet  = {23, 42, 65, 57, 78, 83, 29}
secondSet = {57, 83, 29, 67, 73, 43, 48}
print("First Set ", firstSet)
print("Second Set ", secondSet)

intersection = firstSet.intersection(secondSet)
print("Intersection is ", intersection)
for item in intersection:
  firstSet.remove(item)

print("First Set after removing common element ", firstSet)

First Set  {65, 42, 78, 83, 23, 57, 29}
Second Set  {67, 73, 43, 48, 83, 57, 29}
Intersection is  {57, 83, 29}
First Set after removing common element  {65, 42, 78, 23}


In [ ]:
##Given two sets, Checks if One Set is Subset or superset of Another Set. if the subset is found delete all elements from that set
firstSet  = {57, 83, 29}
secondSet = {57, 83, 29, 67, 73, 43, 48}

print("First Set ", firstSet)
print("Second Set ", secondSet)

print("First set is subset of second set -", firstSet.issubset(secondSet))
print("Second set is subset of First set - ", secondSet.issubset(firstSet))

print("First set is Super set of second set - ", firstSet.issuperset(secondSet))
print("Second set is Super set of First set - ", secondSet.issuperset(firstSet))

if(firstSet.issubset(secondSet)):
  firstSet.clear()
  
if(secondSet.issubset(firstSet)):
  secondSet.clear()
print("First Set ", firstSet)
print("Second Set ", secondSet)

First Set  {57, 83, 29}
Second Set  {67, 73, 43, 48, 83, 57, 29}
First set is subset of second set - True
Second set is subset of First set -  False
First set is Super set of second set -  False
Second set is Super set of First set -  True
First Set  set()
Second Set  {67, 73, 43, 48, 83, 57, 29}


In [ ]:
"""
Write a program that accepts a comma separated sequence of words as input and prints the words in a comma-separated sequence after sorting them alphabetically.
Suppose the following input is supplied to the program:
without,hello,bag,world
Then, the output should be:
bag,hello,without,world
"""
r_input='without,hello,bag,world'

items=[x for x in r_input.split(',')]
items.sort()
print (','.join(items))

bag,hello,without,world


In [ ]:
##Write a program that accepts a sequence of whitespace separated words as input and prints the words after removing all duplicate words and sorting them alphanumerically.
##Suppose the following input is supplied to the program:
##hello world and practice makes perfect and hello world again
##Then, the output should be:
##again and hello makes perfect practice world

s = 'hello world and practice makes perfect and hello world again'
words = [word for word in s.split(" ")]
print(" ".join(sorted(list(set(words)))))

again and hello makes perfect practice world


In [ ]:
#Write a program that accepts a sentence and calculate the number of letters and digits.
#Suppose the following input is supplied to the program:
#hello world! 123
#Then, the output should be:
##LETTERS 10
#DIGITS 3

s = 'mai to raste se ja raha tha #nh247'
d={"DIGITS":0, "LETTERS":0}
for c in s:
    if c.isdigit():
        d["DIGITS"]+=1
    elif c.isalpha():
        d["LETTERS"]+=1
    else:
        pass
print ("LETTERS", d["LETTERS"])
print ("DIGITS", d["DIGITS"])

LETTERS 23
DIGITS 3


In [ ]:
#write a code which uses a list comprehension to square each odd number in a list. The list is input by a sequence of comma-separated numbers.

values = '1,2,3,4,5,6,7,8,9'
numbers = [x for x in values.split(",") if int(x)%2!=0]
print(",".join(numbers))

1,3,5,7,9


In [ ]:
#Write a program to compute the frequency of the words from the input. The output should output after sorting the key alphanumerically. 

raw_input='New to Python or choosing between Python 2 and Python 3? Read Python 2 or Python 3.'

freq = {}   # frequency of words in text
line = raw_input
for word in line.split():
    freq[word] = freq.get(word,0)+1

words = freq.keys()

for w in words:
    print ("%s:%d" % (w,freq[w]))

New:1
to:1
Python:5
or:2
choosing:1
between:1
2:2
and:1
3?:1
Read:1
3.:1


In [ ]:

#Define a function that can accept two strings as input and print the string with maximum length in console. 
#If two strings have the same length, then the function should print al l strings line by line.
def printValue(s1,s2):
	len1 = len(s1)
	len2 = len(s2)
	if len1>len2:
		print (s1)
	elif len2>len1:
		print (s2)
	else:
		print (s1)
		print (s2)
		
printValue("one","three")

three


In [ ]:
#Define a function which can generate a dictionary where the keys are numbers between 1 and 20 (both included) and the values are square of keys. The function should just print the values only.

def printDict():
	d=dict()
	for i in range(1,21):
		d[i]=i**2
	for (k,v) in d.items():	
		print( v)
		

printDict()

1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400


In [ ]:

#Write a program which can filter even numbers in a list by using filter function. The list is: [1,2,3,4,5,6,7,8,9,10].

li = [1,2,3,4,5,6,7,8,9,10]
evenNumbers = filter(lambda x: x%2==0, li)
print(evenNumbers)


In [ ]:
#Define a class named Circle which can be constructed by a radius. The Circle class has a method which can compute the area. 

class Circle(object):
    def __init__(self, r):
        self.radius = r

    def area(self):
        return self.radius**2*3.14

aCircle = Circle(2)
print (aCircle.area())


12.56


In [ ]:
#Define a class named Rectangle which can be constructed by a length and width. The Rectangle class has a method which can compute the area. 

class Rectangle(object):
    def __init__(self, l, w):
        self.length = l
        self.width  = w

    def area(self):
        return self.length*self.width

aRectangle = Rectangle(2,10)
print(aRectangle.area())

20


In [ ]:
#Write a function to compute 5/0 and use try/except to catch the exceptions.

def throws():
    return 5/0

try:
    throws()
except ZeroDivisionError:
    print ("division by zero!")
except Exception:
    print ('Caught an exception')
finally:
    print ('In finally block for cleanup')


division by zero!
In finally block for cleanup


In [ ]:
#Assuming that we have some email addresses in the "username@companyname.com" format, please write program to print the company name of a given email address. Both user names and company names are composed of letters only.


raw_input='user@company.com'

import re
emailAddress = raw_input
pat2 = "(\w+)@(\w+)\.(com)"
r2 = re.match(pat2,emailAddress)
print (r2.group(2))

company


In [ ]:
#Write a program which accepts a sequence of words separated by whitespace as input to print the words composed of digits only.

raw_input='7 bad habbits of 9 ineffective people, 1 should be procrastination'
import re
s = raw_input
print(re.findall("\d+",s))

['7', '9', '1']


In [ ]:
#Please write a binary search function which searches an item in a sorted list. The function should return the index of element to be searched in the list.

import math
def bin_search(li, element):
    bottom = 0
    top = len(li)-1
    index = -1
    while top>=bottom and index==-1:
        mid = int(math.floor((top+bottom)/2.0))
        if li[mid]==element:
            index = mid
        elif li[mid]>element:
            top = mid-1
        else:
            bottom = mid+1

    return index

li=[2,5,7,9,11,17,222]
print (bin_search(li,11))
print (bin_search(li,12))

4
-1


In [ ]:
#Please generate a random float where the value is between 10 and 100 using Python math module.
import random
print (random.random()*100)

43.452726663473065


In [ ]:
#Please write a program to compress and decompress the string "hello world!hello world!hello world!hello world!".
import zlib
s = "hello world!hello world!hello world!hello world!"
t = zlib.compress(s)
print (t)
print (zlib.decompress(t))

print (random.choice([i for i in range(11) if i%2==0])

In [ ]:
# write a program which accepts a string from console and print it in reverse order.
raw_input='my name is, well whats there in the name '

s=raw_input
s = s[::-1]
print (s)

 eman eht ni ereht stahw llew ,si eman ym


In [ ]:
#write a program which prints all permutations of [1,2,3]

import itertools
print (list(itertools.permutations([1,2,3])))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]


In [ ]:
#Write a funtion  which can compute the factorial of a given numbers.

def fact(x):
    if x == 0:
        return 1
    return x * fact(x - 1)

x=int(9)
fact(x)

362880

In [ ]:
#Write a program that accepts a sentence and calculate the number of upper case letters and lower case letters.

s = 'This Is tHe SEntencE'
d={"UPPER CASE":0, "LOWER CASE":0}
for c in s:
    if c.isupper():
        d["UPPER CASE"]+=1
    elif c.islower():
        d["LOWER CASE"]+=1
    else:
        pass
print ("UPPER CASE", d["UPPER CASE"])
print ("LOWER CASE", d["LOWER CASE"])

UPPER CASE 6
LOWER CASE 11


In [ ]:
"""
A website requires the users to input username and password to register. Write a program to check the validity of password input by users.
Following are the criteria for checking the password:
1. At least 1 letter between [a-z]
2. At least 1 number between [0-9]
1. At least 1 letter between [A-Z]
3. At least 1 character from [$#@]
4. Minimum length of transaction password: 6
5. Maximum length of transaction password: 12
Your program should accept a sequence of comma separated passwords and will check them according to the above criteria. Passwords that match the criteria are to be printed, each separated by a comma.
Example
If the following passwords are given as input to the program:
ABd1234@1,a F1#,2w3E*,2We3345
Then, the output of the program should be:
ABd1234@1
"""
r_input='Wak3n@9ends'
import re
value = []
items=[x for x in r_input.split(',')]
for p in items:
    if len(p)<6 or len(p)>12:
        continue
    else:
        pass
    if not re.search("[a-z]",p):
        continue
    elif not re.search("[0-9]",p):
        continue
    elif not re.search("[A-Z]",p):
        continue
    elif not re.search("[$#@]",p):
        continue
    elif re.search("\s",p):
        continue
    else:
        pass
    value.append(p)
print (",".join(value))
#---------------

Wak3n@9ends


In [ ]:
#Define a class Person and its two child classes: Male and Female. All classes have a method "getGender" which can print "Male" for Male class and "Female" for Female class.

class Person(object):
    def getGender(self):
        return "Unknown"


class Male(Person):
    def getGender(self):
        return "Male"


class Female(Person):
    def getGender(self):
        return "Female"


aMale = Male()
aFemale = Female()
print(aMale.getGender())
print(aFemale.getGender())

Male
Female


In [ ]:
## Write a program to find the resolution of an image.
def jpeg_res(filename):
   """"This function prints the resolution of the jpeg image file passed into it"""

   # open image for reading in binary mode
   with open(filename,'rb') as img_file:

       # height of image (in 2 bytes) is at 164th position
       img_file.seek(163)

       # read the 2 bytes
       a = img_file.read(2)

       # calculate height
       height = (a[0] << 8) + a[1]

       # next 2 bytes is width
       a = img_file.read(2)

       # calculate width
       width = (a[0] << 8) + a[1]

   print("The resolution of the image is",width,"x",height)

jpeg_res("img1.jpg")

In [ ]:
## Create a stop watch using python.
# importing the time module 
import time

print("Press ENTER to start the stopwatch")
print("and, press CTRL + C to stop the stopwatch")

# infinite loop
while True:
    try:
        input() #For ENTER
        start_time = time.time()
        print("Stopwatch started...")
        
    except KeyboardInterrupt:
        print("Stopwatch stopped...")
        end_time = time.time()
        print("The total time:", round(end_time - start_time, 2),"seconds")
        break # breaking the loop

In [ ]:
## Write a python function to find the standard deviation.
def stdv(X):
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot = tot + (x - mean)**2
    return (tot/len(X))**0.5

# main code
#  a simple data-set 
sample = [1, 2, 3, 4, 5] 
print("Standard Deviation of the sample is: ", stdv(sample))

sample = [1, 2, 3, -4, -5] 
print("Standard Deviation of the sample is: ", stdv(sample))

sample = [10, -20, 30, -40, 50] 
print("Standard Deviation of the sample is: ", stdv(sample))

Standard Deviation of the sample is:  1.4142135623730951
Standard Deviation of the sample is:  3.2619012860600183
Standard Deviation of the sample is:  32.61901286060018


In [ ]:
## Write a function to find the variance.
def variance(X):
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot = tot + (x - mean)**2
    return tot/len(X)

# main code
#  a simple data-set 
sample = [1, 2, 3, 4, 5] 
print("variance of the sample is: ", variance(sample))

sample = [1, 2, 3, -4, -5] 
print("variance of the sample is: ", variance(sample))

sample = [10, -20, 30, -40, 50] 
print("variance of the sample is: ", variance(sample))

variance of the sample is:  2.0
variance of the sample is:  10.64
variance of the sample is:  1064.0


In [ ]:
## Write a code to find the BMI given weight and height.
# getting input from the user and assigning it to user

height = float(2)
weight = float(60)

# the formula for calculating bmi

bmi = weight/(height**2) 
# ** is the power of operator i.e height*height in this case

print("Your BMI is: {0} and you are: ".format(bmi), end='')

#conditions
if ( bmi < 16):
   print("severely underweight")

elif ( bmi >= 16 and bmi < 18.5):
   print("underweight")

elif ( bmi >= 18.5 and bmi < 25):
   print("Healthy")

elif ( bmi >= 25 and bmi < 30):
   print("overweight")

elif ( bmi >=30):
   print("severely overweight")

Your BMI is: 15.0 and you are: severely underweight


In [ ]:
## Write a code to get the total bits in a number.
# count total number of bits in a number 

# declare a number
num = 61

# use bin () method to get the binary value of a number 
# print binary value output will be 0b111101
print ("binary value of {0} is: {1}".format (num, bin(num)))

# store the length of the binary number
length = len(bin(num))

# since binary value contains 0b as prefix
# so to get exact length total number of bits
# subtract 2 from the length
length -=2

# print length
print ("total number of bits: ", length)

binary value of 61 is: 0b111101
total number of bits:  6


In [ ]:
## find the palindrome number goven a list.
# Give size of list
n='12,13,121,666,7676,98'

# Give list of numbers having size n
l=list(map(int,n.strip().split(',')))

print("Palindrome numbers are:")
# check through the list to check 
# number is palindrome or not
for i in l:
    num=str(i)
    if("".join(reversed(num))==num):
        print(i)

Palindrome numbers are:
121
666


In [ ]:
##Write a code  to find the sum of all numbers below 1000 which are multiples of 3 or 5 in the Python programming language?
# initialize the value of n
n=1000 
# initialize value of s is zero.
s=0 

# checking the number is divisible by 3 or 5
# and find their sum
for k in range(1,n+1):
    if k%3==0 or k%5==0: #checking condition 
        s+=k

# printing the result
print('The sum of the number:',s)

The sum of the number: 234168


In [ ]:
##write a code to  remove hyper link from text.
import re
tweet='https://google.com this is the best '
tweet = re.sub(r'https?:\/\/.\S+', "", tweet) 
print(tweet)

 this is the best 


In [ ]:
## Write a code to remove hashtags.
tweet = re.sub(r'#', '', tweet) 
print(tweet)

 this is the best 


In [ ]:
##write a code to remove old style retweet text "RT" 
tweet = re.sub(r'^RT[\s]+', '', tweet) 
print(tweet)

 this is the best 


In [ ]:
## write a code to replace the slangs with actual value.
#dictionary consisting of the contraction and the actual value 
Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will", 
           "'d":" would","'ve":" have","'re":" are"} 
  
#replace the contractions 
for key,value in Apos_dict.items(): 
    if key in tweet: 
        tweet=tweet.replace(key,value) 
  
print("After Contraction replacement the tweet is:-\n{}".format(tweet))

After Contraction replacement the tweet is:-
 this is the best 


In [ ]:
## Wite a code to split the attached words 
import re 
#separate the words 
tweet = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",tweet) if s]) 
print("After spliting attached words the tweet is:-\n{}".format(tweet))


After spliting attached words the tweet is:-
 this is the best 


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
## write a code to remove punctuations.
#for string operations 
import string           
clean_tweet=[] 
tweet_list=['here','comes' ,'the' ,'#', 'rain', ',' ,'agin' ,'fallin' ,'from' ,'sky' ,'!']
#remove punctuations 
for word in tweet_list: 
    #print(word)
    if word not in string.punctuation: 
        clean_tweet.append(word) 
  
print("clean_tweet = {}".format(clean_tweet))

clean_tweet = ['here', 'comes', 'the', 'rain', 'agin', 'fallin', 'from', 'sky']


In [ ]:
## Wite a code to have 3 numbers are print all the possible combinations.
a=99
b=5
c=121
d=[]
d.append(a)
d.append(b)
d.append(c)
for i in range(0,3):
    for j in range(0,3):
        for k in range(0,3):
            if(i!=j&j!=k&k!=i):
                print(d[i],d[j],d[k])

99 5 121
99 121 5
5 99 121
5 121 99
121 99 5
121 5 99


In [ ]:
## Wite a program to print identity matrix.
n=5
for i in range(0,n):
    for j in range(0,n):
        if(i==j):
            print("1",sep=" ",end=" ")
        else:
            print("0",sep=" ",end=" ")
    print()

1 0 0 0 0 
0 1 0 0 0 
0 0 1 0 0 
0 0 0 1 0 
0 0 0 0 1 


In [ ]:
##write a code to see if the number is an armstrong number or not.
n=1
a=list(map(int,str(n)))
b=list(map(lambda x:x**3,a))
if(sum(b)==n):
    print("The number is an armstrong number. ")
else:
    print("The number isn't an arsmtrong number. ")

The number is an armstrong number. 


In [ ]:
## wite a code to compute a polynomial equation given that the coefficients of the polynomial are stored in a list.
import math
#print("Enter the coefficients of the form ax^3 + bx^2 + cx + d")
lst=[]
for i in range(0,4):
    a=5
    lst.append(a)
x=3
sum1=0
j=3
for i in range(0,3):
    while(j>0):
        sum1=sum1+(lst[i]*math.pow(x,j))
        break
    j=j-1
sum1=sum1+lst[3]
print("The value of the polynomial is:",sum1)

The value of the polynomial is: 200.0


In [ ]:
## Wite a function to print the sum of sine series, run the function with user inputs.
import math
def sin(x,n):
    sine = 0
    for i in range(n):
        sign = (-1)**i
        pi=22/7
        y=x*(pi/180)
        sine = sine + ((y**(2.0*i+1))/math.factorial(2*i+1))*sign
    return sine
x=int(input("Enter the value of x in degrees:"))
n=int(input("Enter the number of terms:"))
print(round(sin(x,n),2))

Enter the value of x in degrees:3
Enter the number of terms:4
0.05


In [ ]:
## wite a code to get the two inputs and print the larger one
string1=input("Enter first string:")
string2=input("Enter second string:")
count1=0
count2=0
for i in string1:
      count1=count1+1
for j in string2:
      count2=count2+1
if(count1<count2):
      print("Larger string is:")
      print(string2)
elif(count1==count2):
      print("Both strings are equal.")
else:
      print("Larger string is:")
      print(string1)

Enter first string:A
Enter second string:bbb
Larger string is:
bbb


In [ ]:
## write a code that tprogram takes a string and creates a dictionary with key as first character and value as words starting with that character.
s="those were the best days of my life"
count = 0
vowels = set("aeiou")
for letter in s:
    if letter in vowels:
        count += 1
print("Count of the vowels is:")
print(count)

Count of the vowels is:
10


In [ ]:
## Write a Python Program to check common letters in the two input strings.
s1=input("Enter first string:")
s2=input("Enter second string:")
a=list(set(s1)&set(s2))
print("The common letters are:")
for i in a:
    print(i)

Enter first string:this is me 
Enter second string:here i am 
The common letters are:
h
e
m
i
 


In [ ]:
## write a code That  takes two strings and displays which letters are in the first string but not in the second string.
s1=input("Enter first string:")
s2=input("Enter second string:")
a=list(set(s1)-set(s2))
print("The letters are:")
for i in a:
    print(i)


Enter first string:here i am 
Enter second string:this isme 
The letters are:
r
a


In [ ]:
#This is a Python Program to display which letters are in the two strings but not in both.

s1=input("Enter first string:")
s2=input("Enter second string:")
a=list(set(s1)^set(s2))
print("The letters are:")
for i in a:
    print(i)

Enter first string:Here I am 
Enter second string:This is me
The letters are:
h
I
T
r
s
H
a
i


In [ ]:
## Wite a function to get the total sum of list recursively.
def sum1(lst):
    total = 0
    for element in lst:
        if (type(element) == type([])):
            total = total + sum1(element)
        else:
            total = total + element
    return total
print( "Sum is:",sum1([[1,2],[3,4]]))

Sum is: 10


In [ ]:
## write a function to get the length of the list recursively.
def length(lst):
    if not lst:
        return 0
    return 1 + length(lst[1::2]) + length(lst[2::2])
a=[1,2,3]
print("Length of the string is: ")
print(a)

Length of the string is: 
[1, 2, 3]


In [ ]:
#Write a code to read from the file.
a=str(input("Enter the name of the file with .txt extension:"))
file2=open(a,'r')
line=file2.readline()
while(line!=""):
    print(line)
    line=file2.readline()
file2.close()

In [ ]:
## Wite a code to  to add two matrices using nested loop

X = [[12,7,3],
    [4 ,5,6],
    [7 ,8,9]]

Y = [[5,8,1],
    [6,7,3],
    [4,5,9]]

result = [[0,0,0],
         [0,0,0],
         [0,0,0]]

# iterate through rows
for i in range(len(X)):
   # iterate through columns
   for j in range(len(X[0])):
       result[i][j] = X[i][j] + Y[i][j]

for r in result:
   print(r)

[17, 15, 4]
[10, 12, 9]
[11, 13, 18]


In [ ]:
# write Program to transpose a matrix using a nested loop

X = [[12,7],
    [4 ,5],
    [3 ,8]]

result = [[0,0,0],
         [0,0,0]]

# iterate through rows
for i in range(len(X)):
   # iterate through columns
   for j in range(len(X[0])):
       result[j][i] = X[i][j]

for r in result:
   print(r)

[12, 4, 3]
[7, 5, 8]


In [ ]:
# Write a program  to multiply two matrices using nested loops

# 3x3 matrix
X = [[12,7,3],
    [4 ,5,6],
    [7 ,8,9]]
# 3x4 matrix
Y = [[5,8,1,2],
    [6,7,3,0],
    [4,5,9,1]]
# result is 3x4
result = [[0,0,0,0],
         [0,0,0,0],
         [0,0,0,0]]

# iterate through rows of X
for i in range(len(X)):
   # iterate through columns of Y
   for j in range(len(Y[0])):
       # iterate through rows of Y
       for k in range(len(Y)):
           result[i][j] += X[i][k] * Y[k][j]

for r in result:
   print(r)

[114, 160, 60, 27]
[74, 97, 73, 14]
[119, 157, 112, 23]


In [ ]:
# Program to sort alphabetically the words form a string provided by the user

my_str = "Hello this Is an Example With cased letters"

# To take input from the user
#my_str = input("Enter a string: ")

# breakdown the string into a list of words
words = [word.lower() for word in my_str.split()]

# sort the list
words.sort()

# display the sorted words

print("The sorted words are:")
for word in words:
   print(word)

The sorted words are:
an
cased
example
hello
is
letters
this
with


In [ ]:
## Wite a # Python program to mail merger
# Names are in the file names.txt
# Body of the mail is in body.txt

# open names.txt for reading
with open("names.txt", 'r', encoding='utf-8') as names_file:

    # open body.txt for reading
    with open("body.txt", 'r', encoding='utf-8') as body_file:

        # read entire content of the body
        body = body_file.read()

        # iterate over names
        for name in names_file:
            mail = "Hello " + name.strip() + "\n" + body

            # write the mails to individual files
            with open(name.strip()+".txt", 'w', encoding='utf-8') as mail_file:
                mail_file.write(mail)

In [ ]:
## Write a finction  to print the resolution of the jpeg image file passed into it

def jpeg_res(filename):

   # open image for reading in binary mode
   with open(filename,'rb') as img_file:

       # height of image (in 2 bytes) is at 164th position
       img_file.seek(163)

       # read the 2 bytes
       a = img_file.read(2)

       # calculate height
       height = (a[0] << 8) + a[1]

       # next 2 bytes is width
       a = img_file.read(2)

       # calculate width
       width = (a[0] << 8) + a[1]

   print("The resolution of the image is",width,"x",height)

jpeg_res("img1.jpg")

In [ ]:
## write a code to   set intersection and difference from two sets to perform different set operations like in mathematics

# define three sets
E = {0, 2, 4, 6, 8};
N = {1, 2, 3, 4, 5};

# set union
print("Union of E and N is",E | N)

# set intersection
print("Intersection of E and N is",E & N)

# set difference
print("Difference of E and N is",E - N)

# set symmetric difference
print("Symmetric difference of E and N is",E ^ N)

Union of E and N is {0, 1, 2, 3, 4, 5, 6, 8}
Intersection of E and N is {2, 4}
Difference of E and N is {0, 8, 6}
Symmetric difference of E and N is {0, 1, 3, 5, 6, 8}


In [ ]:
## write a code to finc the factorial of a given number.
# Python program to find the factorial of a number provided by the user.

# change the value for a different result
num = 7

# To take input from the user
#num = int(input("Enter a number: "))

factorial = 1

# check if the number is negative, positive or zero
if num < 0:
   print("Sorry, factorial does not exist for negative numbers")
elif num == 0:
   print("The factorial of 0 is 1")
else:
   for i in range(1,num + 1):
       factorial = factorial*i
   print("The factorial of",num,"is",factorial)

The factorial of 7 is 5040


In [ ]:
# write a code  to Convert Decimal to Binary, Octal and Hexadecimal.
# Python program to convert decimal into other number systems
dec = 344

print("The decimal value of", dec, "is:")
print(bin(dec), "in binary.")
print(oct(dec), "in octal.")
print(hex(dec), "in hexadecimal.")

The decimal value of 344 is:
0b101011000 in binary.
0o530 in octal.
0x158 in hexadecimal.


In [ ]:
# # write a code  to find the ASCII value of the given character

c = 'p'
print("The ASCII value of '" + c + "' is", ord(c))

The ASCII value of 'p' is 112


In [ ]:
# write a  Python program to swap two variables

x = 5
y = 10

# To take inputs from the user
#x = input('Enter value of x: ')
#y = input('Enter value of y: ')

# create a temporary variable and swap the values
temp = x
x = y
y = temp

print('The value of x after swapping: {}'.format(x))
print('The value of y after swapping: {}'.format(y))

The value of x after swapping: 10
The value of y after swapping: 5


In [ ]:
## Write a  program to shuffle a deck of card

# importing modules
import itertools, random

# make a deck of cards
deck = list(itertools.product(range(1,14),['Spade','Heart','Diamond','Club']))

# shuffle the cards
random.shuffle(deck)

# draw five cards
print("You got:")
for i in range(5):
   print(deck[i][0], "of", deck[i][1])

You got:
13 of Spade
10 of Diamond
6 of Club
1 of Spade
2 of Club


In [ ]:
# write a python code to display the powers of 2 using anonymous function

terms = 10

# Uncomment code below to take input from the user
# terms = int(input("How many terms? "))

# use anonymous function
result = list(map(lambda x: 2 ** x, range(terms)))

print("The total terms are:",terms)
for i in range(terms):
   print("2 raised to power",i,"is",result[i])

The total terms are: 10
2 raised to power 0 is 1
2 raised to power 1 is 2
2 raised to power 2 is 4
2 raised to power 3 is 8
2 raised to power 4 is 16
2 raised to power 5 is 32
2 raised to power 6 is 64
2 raised to power 7 is 128
2 raised to power 8 is 256
2 raised to power 9 is 512


In [ ]:
# Wite a code to print the Dot product of two vectors and the computation time
import time 
import numpy 
import array 
  
# 8 bytes size int 
a = array.array('q') 
for i in range(100000): 
    a.append(i)
  
b = array.array('q') 
for i in range(100000, 200000): 
    b.append(i) 
  
# classic dot product of vectors implementation  
tic = time.process_time() 
dot = 0.0
for i in range(len(a)): 
      dot += a[i] * b[i] 
  
toc = time.process_time() 
  
print("dot_product = "+ str(dot))
print("Computation time = " + str(1000*(toc - tic )) + "ms") 
   
  
n_tic = time.process_time() 
n_dot_product = numpy.dot(a, b) 
n_toc = time.process_time() 
  
print("\nn_dot_product = "+str(n_dot_product)) 
print("Computation time = "+str(1000*(n_toc - n_tic ))+"ms") 

dot_product = 833323333350000.0
Computation time = 29.79149399999992ms

n_dot_product = 833323333350000
Computation time = 0.7593289999996422ms


In [ ]:
# Write a code to print the outer product of two vectors.
# Outer product 
import time 
import numpy 
import array 
  
a = array.array('i') 
for i in range(200): 
    a.append(i); 
  
b = array.array('i') 
for i in range(200, 400): 
    b.append(i) 
  
# classic outer product of vectors implementation  
tic = time.process_time() 
outer_product = numpy.zeros((200, 200)) 
  
for i in range(len(a)): 
   for j in range(len(b)): 
      outer_product[i][j]= a[i]*b[j] 
  
toc = time.process_time() 
  
print("outer_product = "+ str(outer_product)); 
print("Computation time = "+str(1000*(toc - tic ))+"ms") 
   
n_tic = time.process_time() 
outer_product = numpy.outer(a, b) 
n_toc = time.process_time() 
  
print("outer_product = "+str(outer_product)); 
print("\nComputation time = "+str(1000*(n_toc - n_tic ))+"ms") 

outer_product = [[    0.     0.     0. ...     0.     0.     0.]
 [  200.   201.   202. ...   397.   398.   399.]
 [  400.   402.   404. ...   794.   796.   798.]
 ...
 [39400. 39597. 39794. ... 78209. 78406. 78603.]
 [39600. 39798. 39996. ... 78606. 78804. 79002.]
 [39800. 39999. 40198. ... 79003. 79202. 79401.]]
Computation time = 22.38043199999984ms
outer_product = [[    0     0     0 ...     0     0     0]
 [  200   201   202 ...   397   398   399]
 [  400   402   404 ...   794   796   798]
 ...
 [39400 39597 39794 ... 78209 78406 78603]
 [39600 39798 39996 ... 78606 78804 79002]
 [39800 39999 40198 ... 79003 79202 79401]]

Computation time = 1.1391970000000917ms


In [ ]:
# Write a code to do element-wise multiplication of two vectors.
import time 
import numpy 
import array 
  
a = array.array('i') 
for i in range(50000): 
    a.append(i); 
  
b = array.array('i') 
for i in range(50000, 100000): 
    b.append(i) 
  
# classic element wise product of vectors implementation  
vector = numpy.zeros((50000)) 
  
tic = time.process_time() 
  
for i in range(len(a)): 
      vector[i]= a[i]*b[i] 
      toc = time.process_time() 
  
print("Element wise Product = "+ str(vector)); 
print("\nComputation time = "+str(1000*(toc - tic ))+"ms") 
   
  
n_tic = time.process_time() 
vector = numpy.multiply(a, b) 
n_toc = time.process_time() 
  
print("Element wise Product = "+str(vector)); 
print("\nComputation time = "+str(1000*(n_toc - n_tic ))+"ms")

Element wise Product = [0.00000000e+00 5.00010000e+04 1.00004000e+05 ... 4.99955001e+09
 4.99970000e+09 4.99985000e+09]

Computation time = 43.27785999999989ms
Element wise Product = [        0     50001    100004 ... 704582713 704732708 704882705]

Computation time = 0.25192100000026585ms


In [ ]:
#Write a Python program to make an iterator that drops elements from the iterable as soon as an element is a positive number.
import itertools as it
def drop_while(nums):
    return it.dropwhile(lambda x : x < 0, nums)
nums = [-1,-2,-3,4,-10,2,0,5,12]
print("Original list: ",nums)
result = drop_while(nums)
print("Drops elements from the iterable when a positive number arises \n",list(result))
#Alternate solution
def negative_num(x):
    return x < 0
def drop_while(nums):
    return it.dropwhile(negative_num, nums)
nums = [-1,-2,-3,4,-10,2,0,5,12]
print("Original list: ",nums)
result = drop_while(nums)
print("Drops elements from the iterable when a positive number arises \n",list(result))

Original list:  [-1, -2, -3, 4, -10, 2, 0, 5, 12]
Drops elements from the iterable when a positive number arises 
 [4, -10, 2, 0, 5, 12]
Original list:  [-1, -2, -3, 4, -10, 2, 0, 5, 12]
Drops elements from the iterable when a positive number arises 
 [4, -10, 2, 0, 5, 12]


In [ ]:
##Write a Python program to create an iterator that returns consecutive keys and groups from an iterable.
import itertools as it
print("Iterate over characters of a string and display\nconsecutive keys and groups from the iterable:")
str1 = 'AAAAJJJJHHHHNWWWEERRRSSSOOIIU'
data_groupby = it.groupby(str1)
for key, group in data_groupby:
    print('Key:', key)
    print('Group:', list(group))    
print("\nIterate over elements of a list and display\nconsecutive keys and groups from the iterable:")
str1 = 'AAAAJJJJHHHHNWWWEERRRSSSOOIIU'    
str1 = [1,2,2,3,4,4,5,5,5,6,6,7,7,7,8]
data_groupby = it.groupby(str1)
for key, group in data_groupby:
    print('Key:', key)
    print('Group:', list(group))

Iterate over characters of a string and display
consecutive keys and groups from the iterable:
Key: A
Group: ['A', 'A', 'A', 'A']
Key: J
Group: ['J', 'J', 'J', 'J']
Key: H
Group: ['H', 'H', 'H', 'H']
Key: N
Group: ['N']
Key: W
Group: ['W', 'W', 'W']
Key: E
Group: ['E', 'E']
Key: R
Group: ['R', 'R', 'R']
Key: S
Group: ['S', 'S', 'S']
Key: O
Group: ['O', 'O']
Key: I
Group: ['I', 'I']
Key: U
Group: ['U']

Iterate over elements of a list and display
consecutive keys and groups from the iterable:
Key: 1
Group: [1]
Key: 2
Group: [2, 2]
Key: 3
Group: [3]
Key: 4
Group: [4, 4]
Key: 5
Group: [5, 5, 5]
Key: 6
Group: [6, 6]
Key: 7
Group: [7, 7, 7]
Key: 8
Group: [8]


In [ ]:
#Write a Python program to split an iterable and generate iterables specified number of times.
import itertools as it
def tee_data(iter, n):
    return it.tee(iter, n)
#List
result = tee_data(['A','B','C','D'], 5)
print("Generate iterables specified number of times:")
for i in result:
    print(list(i))

#String
result = tee_data("Python itertools", 4)
print("\nGenerate iterables specified number of times:")
for i in result:
    print(list(i))

Generate iterables specified number of times:
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'D']

Generate iterables specified number of times:
['P', 'y', 't', 'h', 'o', 'n', ' ', 'i', 't', 'e', 'r', 't', 'o', 'o', 'l', 's']
['P', 'y', 't', 'h', 'o', 'n', ' ', 'i', 't', 'e', 'r', 't', 'o', 'o', 'l', 's']
['P', 'y', 't', 'h', 'o', 'n', ' ', 'i', 't', 'e', 'r', 't', 'o', 'o', 'l', 's']
['P', 'y', 't', 'h', 'o', 'n', ' ', 'i', 't', 'e', 'r', 't', 'o', 'o', 'l', 's']


In [ ]:
#Write a Python program to generate combinations of a given length of given iterable.

import itertools as it
def tee_data(iter, n):
    return it.tee(iter, n)
#List
result = tee_data(['A','B','C','D'], 5)
print("Generate iterables specified number of times:")
for i in result:
    print(list(i))

#String
result = tee_data("Python itertools", 4)
print("\nGenerate iterables specified number of times:")
for i in result:
    print(list(i))

Generate iterables specified number of times:
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'D']

Generate iterables specified number of times:
['P', 'y', 't', 'h', 'o', 'n', ' ', 'i', 't', 'e', 'r', 't', 'o', 'o', 'l', 's']
['P', 'y', 't', 'h', 'o', 'n', ' ', 'i', 't', 'e', 'r', 't', 'o', 'o', 'l', 's']
['P', 'y', 't', 'h', 'o', 'n', ' ', 'i', 't', 'e', 'r', 't', 'o', 'o', 'l', 's']
['P', 'y', 't', 'h', 'o', 'n', ' ', 'i', 't', 'e', 'r', 't', 'o', 'o', 'l', 's']
